# Ultrametric Matrix Distance: Patient Comparison

Compute ultrametric matrix distances (euclidean, cosine, correlation) across phases for every patient, per EEG band.
Plots show, for each band and metric, phase-by-phase distance matrices side-by-side for all patients.

In [ ]:
%matplotlib inline
#
from lrgsglib import *
#
move_to_rootf(pathname='lrg_eegfc')
#
from lrg_eegfc import *
# Figure output path
path_figs = Path('data') / 'figures' / 'new_ultrametric_distance_pat_comparison'
path_figs.mkdir(parents=True, exist_ok=True)
#
phase_labels = list(PHASE_LABELS)
bands = list(BRAIN_BANDS.keys())
patients = list(PATIENTS_LIST)
metrics = ['euclidean', 'cosine', 'correlation']
#
correlation_protocol = dict(filter_type='abs', spectral_cleaning=False)
#
print(f'Patients: {patients}')
print(f'Phases: {phase_labels}')
print(f'Bands: {bands}')


## Load Data

In [ ]:
data_dict, int_label_map = load_data_dict()
print('✓ Data loaded')

## Build Ultrametric Matrices per Patient, Phase, Band

In [ ]:
def compute_ultrametrics_for_patient(patient: str, filter_order: int = 1):
    """
    For a given patient, compute ultrametric matrices per (phase, band).
    Returns a nested dict: U[band][phase] -> np.ndarray or None.
    """
    U = {b: {} for b in bands}

    # Labels for nodes (channel labels)
    pin_labels = int_label_map[patient]['label']

    for phase in phase_labels:
        data_pat_phase = data_dict[patient][phase]
        data_pat_phase_ts = data_pat_phase['data']
        fs_raw = data_pat_phase.get('fs', None)
        fs = float(np.asarray(fs_raw).flat[0]) if fs_raw is not None else 1000.0

        for band in bands:
            G, label_dict, lnkgM, clTh, corr_mat, dists = process_network_for_phase(
                data_pat_phase_ts, fs, band, correlation_protocol, pin_labels, filter_order=filter_order
            )
            if lnkgM is None or G is None:
                U[band][phase] = None
                continue
            try:
                U[band][phase] = extract_ultrametric_matrix(lnkgM, G.number_of_nodes())
            except Exception as e:
                print(f'[WARN] {patient} {phase} {band}: failed to extract ultrametric ({e})')
                U[band][phase] = None
    return U

# Compute for all patients
ultra_by_pat = {}  # ultra_by_pat[patient][band][phase] -> U matrix or None
for pat in patients:
    print(f'Computing ultrametrics for {pat} ...')
    ultra_by_pat[pat] = compute_ultrametrics_for_patient(pat)
print('✓ Ultrametrics computed for all patients')

## Phase Distance Matrices per Patient, Band, Metric

In [ ]:
# results[metric][band][patient] -> (n_phase x n_phase) matrix
results = {m: {b: {} for b in bands} for m in metrics}
n = len(phase_labels)

for metric in metrics:
    print(f'Metric: {metric}')
    for band in bands:
        for pat in patients:
            dm = np.full((n, n), np.nan, dtype=float)
            for i, pi in enumerate(phase_labels):
                for j, pj in enumerate(phase_labels):
                    if i == j:
                        dm[i, j] = 0.0
                        continue
                    U1 = ultra_by_pat[pat][band].get(pi)
                    U2 = ultra_by_pat[pat][band].get(pj)
                    if U1 is None or U2 is None or U1.shape != U2.shape:
                        dm[i, j] = np.nan
                    else:
                        dm[i, j] = ultrametric_matrix_distance(U1, U2, metric=metric)
            results[metric][band][pat] = dm
    print('  ✓ done')
print('✓ All phase-distance matrices computed')

## Plot: Side-by-Side Patients per Band and Metric

In [ ]:
def plot_band_metric_side_by_side(metric: str, band: str):
    # Determine shared color scale across patients (ignore NaNs)
    vmax = 0.0
    for pat in patients:
        m = results[metric][band].get(pat)
        if m is not None and np.isfinite(m).any():
            vmax = max(vmax, np.nanmax(m))
    if vmax == 0.0:
        print(f'[SKIP] No finite values for {metric} / {band}')
        return

    fig, axes = plt.subplots(1, len(patients), figsize=(4*len(patients)+2, 4), constrained_layout=True)
    if len(patients) == 1:
        axes = [axes]

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='lightgray')

    for ax, pat in zip(axes, patients):
        M = results[metric][band][pat]
        im = ax.imshow(M, vmin=0.0, vmax=vmax, cmap=cmap, aspect='equal')
        ax.set_title(f"{pat}")
        ax.set_xticks(range(len(phase_labels)))
        ax.set_yticks(range(len(phase_labels)))
        ax.set_xticklabels(phase_labels, rotation=45, ha='right')
        ax.set_yticklabels(phase_labels)
        for spine in ax.spines.values():
            spine.set_visible(False)

    cbar = fig.colorbar(im, ax=axes, shrink=0.85)
    cbar.set_label(f'{metric} distance')
    band_label = BRAIN_BAND_LABELS.get(band, band)

    outdir = path_figs / metric
    outdir.mkdir(parents=True, exist_ok=True)
    outfile = outdir / f'{band}.png'
    fig.savefig(outfile, dpi=200, bbox_inches='tight')
    plt.show()
    print(f'Saved: {outfile}')

# Generate all plots
for metric in metrics:
    for band in bands:
        plot_band_metric_side_by_side(metric, band)

print('✓ All figures generated')


In [ ]:
patient = 'Pat_03'
band = 'beta'
filter_order = 1
#
store_dict = {}
for phase in phase_labels:
    pin_labels = int_label_map[patient]['label']
    data_pat_phase = data_dict[patient][phase]
    data_pat_phase_ts = data_pat_phase['data']
    fs_raw = data_pat_phase.get('fs', None)
    fs = float(np.asarray(fs_raw).flat[0]) if fs_raw is not None else 1000.0
    G, label_dict, lnkgM, clTh, corr_mat, dists = process_network_for_phase(
        data_pat_phase_ts, fs, band, correlation_protocol, pin_labels, filter_order=filter_order
    )
    store_dict[phase] = {
        'G': G,
        'lnkgM': lnkgM,
        'clTh': clTh,
        'corr_mat': corr_mat,
        'dists': dists
    }

In [ ]:
fig, axes = plt.subplots(3, len(phase_labels), figsize=(5*len(phase_labels)+2, 16), constrained_layout=True)

for phase in phase_labels:
    ax_row = 0
    ax_col = phase_labels.index(phase)
    ax_corr = axes[ax_row, ax_col]
    ax_ultra = axes[ax_row+1, ax_col]
    ax_dendro = axes[ax_row+2, ax_col]

    data_phase = store_dict[phase]
    corr_mat = data_phase['corr_mat']
    lnkgM = data_phase['lnkgM']

    im1 = ax_corr.imshow(corr_mat, vmin=0.0, vmax=1.0, cmap='viridis', aspect='equal')
    ax_corr.set_title(f'Corr: {phase}')
    ax_corr.set_xticks([])
    ax_corr.set_yticks([])
    pos = nx.spring_layout(store_dict[phase]['G'], seed=42, k=0.05)
    nx.draw(store_dict[phase]['G'], ax=ax_ultra, pos=pos, node_size=20, node_color='blue', with_labels=False)
    lnkgM = store_dict[phase]['lnkgM']
    res, *_ = compute_optimal_threshold(lnkgM)
    dendro = dendrogram(lnkgM, ax=ax_dendro, color_threshold=res, above_threshold_color='k')
    tmin = lnkgM[:, 2][0] * 0.8
    tmax = lnkgM[:, 2][-1] * 1.01
    ax_dendro.set_ylim(tmin, tmax)
    ax_dendro.set_yscale('log')